<a href="https://colab.research.google.com/github/Fish210/Alliance-Optimization-Model/blob/main/Alliance_Optimization_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
##### Imports #####
!pip install catboost
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

##### Load in Input Data #####
PATH = "/content/FTC_ML_Input_ActionsPlusScores_15Teams_FILLED_ebh3.xlsx"
df = pd.read_excel(PATH, sheet_name="team_match_actions")
df = df[df["team"].notna()].copy()

count_cols=[
    "auto_leave", "auto_artifact_cl_count", "auto_artifact_overflow_count", "auto_motif_match_count",
    "tele_artifact_cl_count", "tele_artifact_overflow_count","tele_motif_match_count", "tele_depot_count"
]
for c in count_cols:
  df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
##### Attatch scores to actions #####
df["auto_points"] = (
    3*df["auto_leave"]
    + 3*df["auto_artifact_cl_count"]
)
df[tele_points](

)

